In [ ]:
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 

import Artha.neodb
from Artha.neodb import Neo
import sqlite3
from tqdm import tqdm
import csv

neo = Neo("neo4j://localhost:7687", "neo4j", "part2")
conn = sqlite3.connect("../data/ucheckra_.db")
conn.row_factory = neo.dict_factory
checkra_follows = conn.cursor().execute("select * from following").fetchall()
print("done")

In [ ]:
print(len(neo.get_nodes()), len(neo.get_relations()))

In [ ]:
neo.clear_db()

In [ ]:
nodes = neo.load_csv_data("checkra_")

print(nodes.data()[0]["count(r)"])
print(len(neo.get_nodes()), len(neo.get_relations()))

In [ ]:
with open("text.txt", "w+") as w:
    for ind, record in enumerate(node_records):
        w.write(str(ind)+" "+str(record["n"]["name"])+"\n")

In [ ]:
with open("text1.txt", "w+") as w:
    for ind, record in enumerate(edge_records):
        w.write(str(ind)+" "+str(record["n"]["name"])+" "+str(record["m"]["name"])+"\n")

In [ ]:
with open("text1.txt", "w+") as w:
    for ind, record in enumerate(edge_records):
        w.write(str(ind)+" "+str(record["r"].nodes[1]["name"])+"\n")

In [ ]:
for i in edge_records:
    print(i["r"].nodes[1]["name"])

In [ ]:
for ind, record in enumerate(edge_records):
    # print(dir(record[""]))
    print(str(ind)+" "+edge_records["r"].nodes[1]["name"])

In [ ]:
conn_temp = sqlite3.connect("../data/users/uBrianRoemmele.db")
conn_temp.row_factory = dict_factory


temp_follows = conn_temp.cursor().execute("select id, name, username from following").fetchall()

    # with open(path, "w+") as csvfile:
    #     writer = csv.DictWriter(csvfile, fieldnames = ["id", "name", "username", "by"])
    #     writer.writeheader()
    #     writer.writerows(temp_follows)

In [ ]:
%%time
temp_follows = conn_temp.cursor().execute("select id, name, username from following").fetchall()

In [ ]:
%%time

num = user_edges("Ninjascalp")
print("done", "Ninjascalp", num)
# for user in tqdm(checkra_follows):
#     num = user_edges(user["username"])
#     print("done", user["username"], num)